In [2]:
import numpy as np
from numpy.lib.format import open_memmap
import myfunctions as mf
import os
from tqdm import tqdm

In [3]:
exp_list = ['P28A_FT_H_Exp4_2', 'P28B_ISC_FT_H_Exp2', 'VCT5_FT_N_Exp1']
crop_list = [100, 50, 100]
parent_dir = '/Volumes/T7/Thesis/'

In [4]:
for exp, crop in zip(exp_list, crop_list):
    exp_dir = os.path.join(parent_dir, exp)
    v1 = open_memmap(os.path.join(exp_dir, 'hypervolume.npy'), mode='r')
    t = v1.shape[0]
    z = v1.shape[1]
    xy = v1.shape[2]
    v2 = open_memmap(os.path.join(exp_dir, 'v2.npy'), mode='w+', dtype=np.half, shape=(t, z, xy-(2*crop), xy-(2*crop)))
    for time in tqdm(range(t), desc=exp):
        v2[time] = v1[time, :, crop:v1.shape[2]-crop, crop:v1.shape[2]-crop]

VCT5_FT_N_Exp1: 100%|██████████| 33/33 [02:05<00:00,  3.81s/it]
